# Data Leakage

data leakage(데이터 누수)가 무엇인지 어떻게 방지할 수 있는지 알아볼 것이다. 만약 이를 어떻게 방지할 것인가를 모른다면, data leakage는 자주 발생할 것이고, 이는 모델을 망칠 수도 있다. 따라서 data leakage라는 개념은 초보 데이터분석가에는 매우 중요한 개념이다.

## 서론

data leakage는 train data에 target에 대한 정보가 포함되어 있지만 그 정보를 실제 예측에서는 사용할 수 없는 경우 발생한다. train set와 valid set에서 좋은 성능을 보이지만, 실제 모델이 수행 될 때는 제대로 작동하지 않을 것이다.

다시 말해서, data leakage가 발생한다면 모델을 실제 사용하여 의사결정을 내리기 전까지 문제점을 찾지 못할 수도 있다.(train, valid 세트에선 성능이 좋으므로) 뒤늦게 모델이 매우 부정확하다는 것을 깨닫게 된다.

data leakage에는 2가지 유형이 있다.
- target leakage(타겟 누수)
- train-test contamination (훈련 테스트 오염) 

### Target Leakage
target leakage는 예측 시점에서 사용할 수 없는 데이터가 포함되어 있을 때 발생한다. 예를 통해 이해해보자.

In [1]:
import pandas as pd

dict = {'got_pneumonia' : [False, False, True], 'age' : [65,72,58], 'weight' : [100,130,100], 'male' : [False, True, False], 'took_antibiotic_medicine' : [False, False, True]}

pd.DataFrame(dict)

,got_pneumonia,age,weight,male,took_antibiotic_medicine
0,False,65,100,False,False
1,False,72,130,True,False
2,True,58,100,False,True


위 데이터를 이용해 누가 폐렴(pneumonia)에 걸렸나를 예측하고자 한다. 

사람들은 폐렴에 걸린 후에 항생제(antibiotic)를 복용한다. 

이때 폐렴 여부와 항생제의 복용 여부 사이엔 강한 관계가 존재한다는 것을 알 수 있다. 

위 데이터를 활용하여 모델을 만들게 된다면, 모델은 ''took_antibiotic_medicine'이 False인 사람은 폐렴에 걸리지 않는다고 예측할 것이다.

폐렴에 걸렸는지를 예측할 것인데 폐렴에 걸린 후의 항생제 복용 여부를 사용한다는 것은 부적절해보인다. 

이런 것이 target leakage다. 

이를 방지하려면 target이 결정된 후의 생성되는 변수들을 데이터세트에서 제외해야한다.

### Train-Test Contamination
train-test contamination는 훈련 데이터와 검증데이터를 제대로 구별하지 않았을 때, 발생한다.

검증 데이터는 모델이 고려하지 않았던 데이터에 대해 얼마나 좋은 성능 보이는가를 측정하는 것이다.

하지만 검증 데이터가 전처리에 영향을 준다면? 위 과정이 변질 될 수 있다.

예를 들어 train_test_split 함수를 쓰기 전에, 전처리(결측치 대체 등)를 한다고 가정하자. 

결과적으로 검증 데이터의 점수는 좋을 것이고, 우리들은 의심없이 이 결과를 신뢰할 것이다. 

하지만 모델이 실제로 사용될 때는 성능이 저하된다. 복잡한 feature engineering을 한다면 이 문제는 더욱 심각해질 것이다.

만약 검증 데이터가 train-test split을 기반으로 만들어졌을 때, 검증 데이터를 모든 fitting에서 제외하고, 전처리 단계의 fitting에 포함시켜야 한다.

사이킷런의 파이프라인을 이용하면 더욱 쉽다. Cross-validation을 사용할 때는 파이프 라인 내에서 전처리를 수행하는 것이 훨씬 더 중요하다.

### 예제
예제에선 target leakage를 발견하고, 제거하는 것을 살펴볼 것이며, 신용카드 어플리케이션 데이터세트를 사용한다. feature들을 통해 신용카드 신청이 승인(1)인지 거절되었는지(0)를 예측해볼 것이다.

In [2]:
# 데이터 불러오기
data = pd.read_csv('data/AER_credit_card_data.csv', true_values = ['yes'], false_values = ['no'])

X = data.drop('card', axis = 1)
y = data['card']

print('행의 개수: ',X.shape[0])
X.head()

행의 개수:  1319


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# 전처리를 안 하기 때문에 pipeline을 사용 안 해도 괜찮다.
pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(pipeline, X, y, cv = 5, scoring = 'accuracy')
print('CV accuary: %f' % cv_scores.mean())

CV accuary: 0.978776


0.98이라는 보기 힘든 수치가 나왔다. 98%라는 정확도는 의심해볼만하다. target leakage가 있는지 살펴보도록 하자!

- card: 1 if credit card application accepted, 0 if not
- reports: Number of major derogatory reports
- age: Age n years plus twelfths of a year
- income: Yearly income (divided by 10,000)
- share: Ratio of monthly credit card expenditure to yearly income
- <b> expenditure: Average monthly credit card expenditure </b>
- owner: 1 if owns home, 0 if rents
- selfempl: 1 if self-employed, 0 if not
- dependents: 1 + number of dependents
- months: Months living at current address
- majorcards: Number of major credit cards held
- active: Number of active credit accounts

몇 개의 변수가 의심스럽다. 예를 들어 expenditure는 발급 신청한 카드의 월별 지출에 대한 평균을 말하는 것인지, 아니면 발급 신청 전의 고객이 사용했던 카드의 월별 지출을 말하는 것인지 헷갈린다.

In [4]:
# 비교해보자
expenditures_cardholders = X['expenditure'][y]
expenditures_noncardholders = X['expenditure'][~y]

print('Fraction of those who did not receive a card and had no expenditures: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Fraction of those who received a card and had no expenditures: %.2f' \
      %(( expenditures_cardholders == 0).mean()))

Fraction of those who did not receive a card and had no expenditures: 1.00
Fraction of those who received a card and had no expenditures: 0.02


이를 보면 카드를 받지 못한 사람은 모두 지출이 없었고, 카드를 받은 사람의 2% 만이 지출이 없었다. 이를 통해 target leakage가 발생했다는 것을 알 수 있었다. 

In [5]:
# target leakage의 가능성이 있는 변수들도 함께 제거
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis = 1)

cv_scores = cross_val_score(pipeline, X2, y, cv = 5, scoring = 'accuracy')
print('CV accuary: %f' % cv_scores.mean())

CV accuary: 0.832437


정확도는 많이 낮아졌다. 하지만 target leakage가 있던 실제 모델을 사용할 시에 이 모델보다 성능이 더 안 좋을 수 있다.

## Reference :
[Intermediate Machine Learning(Kaggle)](https://www.kaggle.com/alexisbcook/data-leakage)